# Demo: Serenity VaR Calculation API

In addition to supporting risk attribution, the latest version of Serenity also supports running VaR calculations
and backtesting those VaR models with your portfolio. This notebook will show you how to select a VaR model and
use it to compute Value at Risk (VaR) for a portfolio constructed from the Serenity security master.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
%run -i init_demo.py

To run a VaR calculation the first thing we need is a portfolio:

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'BTC': 100,
    'ETH': 1000,
    'SOL': 10000,
    'BNB': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

The next thing we need is a model configuration. IMPORTANT: this feature is not yet supported in production, but we are including it here because this is the pattern you will need to follow going forward:

In [ ]:
from serenity_sdk.types import CalculationContext

model_short_name = 'risk.var.parametric.normal'  
model_meta = api.model().load_model_metadata(datetime.date.today())
model_config_id = model_meta.get_model_configuration_id(model_short_name)

Now that we have it we can create the `CalculationContext`, the same as with risk attribution. We will set an as-of-date and pass our parametric VaR model configuration ID. We then run VaR for a given date:

In [ ]:
from datetime import date

# construct the input parameters for VaR
ctx = CalculationContext(as_of_date=date(2020, 5, 1), model_config_id=model_config_id)

# run the VaR calc
result = api.risk().compute_var(ctx, portfolio)
result

We can also run a backtest:

In [ ]:
result = api.risk().compute_var_backtest(ctx, portfolio, date(2020, 5, 1), date(2022, 5, 1), quantiles=[1, 5, 95, 99])

In [ ]:
rows = [{
    'runDate': result.run_date,
    'baseline': result.baseline,
    'varAbsolute': result.quantiles[3].var_absolute,
    'varRelative': result.quantiles[3].var_relative
} for result in result.results]
backtest_df = pd.DataFrame(rows)
backtest_df.set_index('runDate', inplace=True)

loss_fmt = lambda val: f'${val:,.2f}' if val >= 0 else f'(${abs(val):,.2f})'
pct_fmt = lambda val: f'({abs(val):,.1%})' if val < 0 else f'{abs(val):,.1%}'
backtest_df.head().style.format({
  'baseline': loss_fmt,
  'varAbsolute': loss_fmt,
  'varRelative': pct_fmt
}).applymap(lambda val: 'color: red', subset=['varRelative'])

And we can identify the breach dates:

In [ ]:
rows = []
for breach in result.breaches:
  for quantile in breach.quantiles:
    if quantile.quantile in [95, 99]:
      rows.append({
          'breachDate': breach.breach_date.date(),
          'portfolioLossAbsolute': breach.portfolio_loss_absolute,
          'portfolioLossRelative': breach.portfolio_loss_relative,
          'quantile': quantile.quantile / 100.0,
          'varLevelAbsolute': quantile.var_absolute,
          'varLevelRelative': quantile.var_relative,
      })
breach_df = pd.DataFrame(rows)
breach_df.style.format({
  'portfolioLossAbsolute': loss_fmt,
  'portfolioLossRelative': pct_fmt,
  'quantile': pct_fmt,
  'varLevelAbsolute': loss_fmt,
  'varLevelRelative': pct_fmt
}).applymap(lambda val: 'color: red', subset=['portfolioLossAbsolute', 'portfolioLossRelative', 'varLevelAbsolute', 'varLevelRelative'])

Now let's do some charting with the backtest data; this is a sneak peek at the kinds of visualizations we will be offering in the Serenity front-end at the end of 2022:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

kind = "Historical Sim"
rows = [{
    'runDate': result.run_date,
    'baseline': result.baseline,
    'VaR_01': result.quantiles[0].var_absolute,
    'VaR_05': result.quantiles[1].var_absolute,
    'VaR_95': result.quantiles[2].var_absolute,
    'VaR_99': result.quantiles[3].var_absolute,
} for result in result.results]
df = pd.DataFrame(rows)
df.set_index('runDate', inplace=True)

fig, ax = plt.subplots(1, 1, figsize=(14, 10))
ax.set_title(f"VaR Backtest ({kind})")
ax.set_ylabel("Price ($)")
ax.set_xlabel("Date")
# convert to pd.Timestamp to date time to ease plotting
df.index = pd.to_datetime(df.index)
ax.plot(df.index, df.baseline, "-", color="purple", label="baseline")
ax.fill_between(
    df.index, df.VaR_99, df.VaR_95, alpha=0.5, color="royalblue", label="95%-99%"
)
ax.fill_between(
    df.index,
    df.VaR_95,
    df.VaR_05,
    alpha=0.5,
    color="lightsteelblue",
    label="05%-95%",
)
ax.fill_between(
    df.index, df.VaR_05, df.VaR_01, alpha=0.5, color="royalblue", label="01%-05%"
)
df[(df.VaR_01 > df.baseline)]["baseline"].plot(
    ax=ax, style="o", color="yellow", label="01% Breach"
)
ax.plot()
ax.legend()